In [ ]:
# Our usual imports
import pandas as pd
import geopandas as gpd
import numpy as np
from numpy import asarray
from matplotlib import pyplot as plt
import os
import pandas as pd
import json
#import pyproj
%matplotlib inline

import ee
import geemap

# Initialize the Earth Engine module.
ee.Initialize()

## For loading a csv 
Here is the function to read and trim a csv dataframe and then convert it to a geopandas data frame from latitude and longitude points. If there is already a geometry column, one merely needs to transform the dataframe into a geopandas dataframe and set the crs. Additionally, if it is in geojson format, scroll down for the function for that.

The key columns that you need for the csv to work is the school ID number (so that we can merge it back to the training dataframe), and the lat and lon points, if you have geometry, that also works)

Disclaimer: Google Earth Engine can only run with a limited amount of rows, around 10,000-15,000 being the limit. Therefore, if you have a dataframe with more than this amount of rows, we suggest limiting the dataframe after this step to a smaller number like 10K in order to run through the rest of the EDA. 

In [ ]:
def df_transformation(filename, keep_columns_list):
    #Read in the df
    df = pd.read_csv(filename)
    #trimming to just the ones we want
    df = df[keep_columns_list]
    #Turning it into a geodataframe, creates another column called geometry. 
    #We need this to convert into Google Earth Feature Collection
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude)).set_crs(epsg=4326)
    return gdf

In [ ]:
#Need a csv with schools in points, lat and lon, and 
filename = '../data/school_loc/school_data_bra.csv'

#list of columns we are choosing to keep
keep_columns_list = ['source_school_id','latitude','longitude']

df = df_transformation(filename, keep_columns_list)
df

In [ ]:
## If the data is over ~15K rows, trim the df here
df = df[:10000]

In [ ]:
df.shape

## Reformatting for GEE
Now we need to reformat our geopandas dataframe to a geojson feature collection that will be processed within the Google Earth Engine code. 

In [ ]:
#Here is the function that turns a geopandas df into a feature collection and then sets a 5 km buffer around the schools
def Feature_Collection(gdf, buffer_amount):
    #Converting geopandas series, the geometry column to json
    ROI = gpd.GeoSeries(gdf['geometry']).to_json()
    # convert dictionary string to dictionary
    res = json.loads(ROI)
    #Converts dictionary object into a feature collection
    Feature_Collection = geemap.geojson_to_ee(res, geodesic=True)
    #Setting a 5 km buffer
    Feature_Collection = Feature_Collection.map(lambda f: f.buffer(buffer_amount))
    return Feature_Collection

In [ ]:
#Running the Feature Collection Function on the dataframe from the previous step, renamed it Feature_collection

#Buffer is set in meters, we set at 5000 meters or 5 km 
buffer_amount = 5000

# turning into feature collection 
schools = Feature_Collection(df, buffer_amount)

# subset for faster preocessing
#schools = schools.limit(100)

## Nighttime Imagery EDA

### Brazil and Thailand Average Radiance for 2014 and 2019

In [ ]:
## First we will show Brazil and Thailand Average Radiance for 2014 and 2019
Brazil = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME', 'Brazil'))
Thailand = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME', 'Thailand'))

Brazil_avg_rad_2014 = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate(
    "2014-07-01","2014-12-31").mean().select('avg_rad')

Brazil_avg_rad_2019 = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate(
    "2019-07-01","2019-12-31").mean().select('avg_rad')

# set some thresholds
vis_params = {
  'min': 0,
  'max': 100,
    'palette': ['#000000',
'#01122b',
'#19568b',
'#ecf3e7',
'#ff0047']
}

colors = vis_params['palette']
vmin = vis_params['min']
vmax = vis_params['max']

# initialize our map
map1 = geemap.Map()
map1.centerObject(Brazil, 5)

#Add various layers, toggle to see different layers in the upper right corner
map1.addLayer(Brazil_avg_rad_2014.clip(schools), vis_params, 'Avg Rad Brazil Schools 2014')
map1.addLayer(Brazil_avg_rad_2019.clip(schools), vis_params, 'Avg Rad Brazil Schools 2019')
map1.addLayer(Brazil_avg_rad_2014.clip(Brazil), vis_params, 'Avg Rad Brazil 2014')
map1.addLayer(Brazil_avg_rad_2019.clip(Brazil), vis_params, 'Avg Rad Brazil 2019')
map1.add_colorbar(vis_params, label="Average Radiance", layer_name="Average Radiance")
map1.addLayerControl()
#uncomment if you want to save an html version of the map
# map1.save("avg_rad_layers.html")
map1

### Cloud Free Coverage

In [ ]:
## First we will show Brazil and Thailand Average Radiance for 2014 and 2019

#Choose if you want Brazil
country =  ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME', 'Brazil'))
#Choose if you want Thailand
# country = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME', 'Thailand'))

#Select which band you want to display, for nighttime the two options are avg_rad (average radiance) or cf_cvg (cloud free coverage)
#band = 'avg_rad'
band = 'cf_cvg'

#Takes the average of light emittance in 2014
band_2014 = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate(
    "2014-07-01","2014-12-31").mean().select(band)

#Takes the average of light emittance in 2014
band_2019 = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate(
    "2019-07-01","2019-12-31").mean().select(band)

# set some  visualization thresholds
#cloud free layer has a much lower maximum than the average radiance
vis_params = {
  'min': 0,
  'max': 15,
    'palette': ['#000000',
'#01122b',
'#19568b',
'#ecf3e7',
'#ff0047']
}

colors = vis_params['palette']
vmin = vis_params['min']
vmax = vis_params['max']


# initialize our map
map1 = geemap.Map()
map1.centerObject(Brazil, 5)
#add each layer and clip it to either the school or country feature collection, establish the visualization parameters and name the layer
map1.addLayer(band_2014.clip(schools), vis_params, band + ' Brazil Schools 2014')
map1.addLayer(band_2019.clip(schools), vis_params, band + ' Brazil Schools 2019')
map1.addLayer(band_2014.clip(country), vis_params, band + ' Brazil 2014')
map1.addLayer(band_2019.clip(country), vis_params, band + ' Brazil 2019')
map1.add_colorbar(vis_params, label="Cloud Free Coverage", layer_name="Cloud Free Coverage")
map1.addLayerControl()
#Save the map to files
map1.save("cf_cvg_layers.html")
map1

## Visualizing the change from 2015 to 2019 in Average Radiance by school points and then by country

In [ ]:
#Select which band you want to display, for nighttime the two options are avg_rad (average radiance) or cf_cvg (cloud free coverage)
band = 'avg_rad'
# band = 'cf_cvg'

# set some  visualization thresholds
#cloud free layer has a much lower maximum than the average radiance
vis_params = {
  'min': 0,
  'max': 100, #When its avg rad, the max should be 100
    'palette': ['#000000',
'#01122b',
'#19568b',
'#ecf3e7',
'#ff0047']
}

colors = vis_params['palette']
vmin = vis_params['min']
vmax = vis_params['max']

viirs2015 = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate(
    "2014-07-01","2014-12-31").mean().select(band).clip(schools) #Here is where you clip it to just the school points
viirs2019 = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate(
    "2019-07-01","2019-12-31").mean().select(band).clip(schools) #Here is where you clip it to just the school points

viirs_15_tile = geemap.ee_tile_layer(viirs2015, vis_params, 'Jul-Dec 2014', opacity=0.75)
viirs_19_tile = geemap.ee_tile_layer(viirs2019, vis_params, 'Jul-Dec 2019', opacity=0.75)

# initialize our map
map2 = geemap.Map()
map2.centerObject(schools, 6)
map2.split_map(left_layer=viirs_15_tile, right_layer=viirs_19_tile)
map2.add_colorbar(vis_params, label="Average Radiance", layer_name="Average Radiance")
map2.addLayerControl()
map2

### Showing the change from 2014 to 2019 on a country level with the average radiance band

In [ ]:
#Choose if you want Brazil
country =  ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME', 'Brazil'))
#Choose if you want Thailand
# country = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME', 'Thailand'))

#Select which band you want to display, for nighttime the two options are avg_rad (average radiance) or cf_cvg (cloud free coverage)
band = 'avg_rad'
# band = 'cf_cvg'

viirs2015 = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate(
    "2014-07-01","2014-12-31").mean().select(band).clip(country) #Clipping it to the country of Brazil instead
viirs2019 = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate(
    "2019-07-01","2019-12-31").mean().select(band).clip(country)

viirs_15_tile = geemap.ee_tile_layer(viirs2015, vis_params, 'Jul-Dec 2014', opacity=0.75)
viirs_19_tile = geemap.ee_tile_layer(viirs2019, vis_params, 'Jul-Dec 2019', opacity=0.75)

# initialize our map
map2 = geemap.Map()
map2.centerObject(country, 5)
map2.split_map(left_layer=viirs_15_tile, right_layer=viirs_19_tile)
map2.add_colorbar(vis_params, label="Average Radiance", layer_name="Average Radiance")
map2.addLayerControl()
map2

### Showing the change from 2014 to 2019 on a country level with the cloud free coverage band

In [ ]:
#Choose if you want Brazil
country =  ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME', 'Brazil'))
#Choose if you want Thailand
# country = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME', 'Thailand'))

#Select which band you want to display, for nighttime the two options are avg_rad (average radiance) or cf_cvg (cloud free coverage)
# band = 'avg_rad'
band = 'cf_cvg'

# set some  visualization thresholds
vis_params = {
  'min': 0,
  'max': 15, #max is at 15 for cf_cvg
  'palette': ['#000000',
'#01122b',
'#19568b',
'#ecf3e7',
'#ff0047']
}

# colors = vis_params['palette']
vmin = vis_params['min']
vmax = vis_params['max']


viirs2015 = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate(
    "2014-07-01","2014-12-31").mean().select(band).clip(country)
viirs2019 = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate(
    "2019-07-01","2019-12-31").mean().select(band).clip(country)

viirs_15_tile = geemap.ee_tile_layer(viirs2015, vis_params, 'Jul-Dec 2014', opacity=0.75)
viirs_19_tile = geemap.ee_tile_layer(viirs2019, vis_params, 'Jul-Dec 2019', opacity=0.75)

# initialize our map
map2 = geemap.Map()
map2.centerObject(country, 5)
map2.split_map(left_layer=viirs_15_tile, right_layer=viirs_19_tile)
map2.add_colorbar(vis_params, label="Cloud Free Coverage", layer_name="Cloud Free Coverage")
map2.addLayerControl()
map2

## Visualizing the rate of change from 2014 to 2020 for Average radiance

In [ ]:
#Visualizing the rate of change from 2014 to 2020
viirs = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate(
    '2014-01-01','2019-12-31')

# sort by image "time_end"
first_img = viirs.sort('system:time_end').first()

# reverse sort so that last=first
last_img = viirs.sort('system:time_end',False).first()

# get rate of change (diff over # months: 73)
viirs_slope = (last_img.select('avg_rad').subtract(first_img.select('avg_rad'))).divide(73).clip(schools)

# get rate of change for cloud free coverage (diff over # months: 73)
viirs_slope_cf_cvg = (last_img.select('cf_cvg').subtract(first_img.select('cf_cvg'))).divide(73).clip(schools)

#Loading Nighttime Image and taking the mean
viirs_first = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate('2014-01-01','2014-12-31').mean()
viirs_last = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate('2019-01-01','2019-12-31').mean()

# get rate of change (diff over # years: 6)
viirs_slope_year = (viirs_last.select('avg_rad').subtract(viirs_first.select('avg_rad'))).divide(5).clip(schools)

In [ ]:
#change min, max to reflect the actual stats
vis_params = {'min':-1,
             'max':1,
             'palette':['1d4877','1b8a5a','f68838','ee3e32']}

colors = vis_params['palette']
vmin = vis_params['min']
vmax = vis_params['max']

# make it opaque so we can see underlying basemap
# initialize our map
viirsMap = geemap.Map()
viirsMap.centerObject(schools, 9)
viirsMap.addLayer(viirs_slope, vis_params, '2014-2020 VIIRS-DNB monthly rate of change Avg Rad',opacity=.75)
viirsMap.addLayer(viirs_slope_cf_cvg, vis_params, '2014-2020 VIIRS-DNB monthly rate of change Cloud Free',opacity=.75)
viirsMap.addLayer(viirs_slope_year, vis_params, '2014-2020 VIIRS-DNB yearly rate of change Avg Rad',opacity=.75)
viirsMap.add_colorbar(vis_params, label="Rate of Change", layer_name="Rate of Change")
viirsMap.addLayerControl()
viirsMap

## Next, we show a map for the Global Human Modification Index

In [ ]:
### Global Human Modification
Brazil = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME', 'Brazil'))

GHM = ee.ImageCollection('CSP/HM/GlobalHumanModification').mean().clip(Brazil)

# set some thresholds
visualization = {"min": 0.0, "max": 1.0, "palette": ['0c0c0c', '071aff', 'ff0000', 'ffbd03', 'fbff05', 'fffdfd']}


# initialize our map
map1 = geemap.Map()
map1.centerObject(Brazil, 6)
map1.addLayer(GHM, visualization, 'Human modification')
map1.add_colorbar(visualization, label="GHM", layer_name="GHM")
map1.addLayerControl()
map1

## Using MODIS to find the Normalized Difference Vegetation Index

In [ ]:
### Normalized Difference Vegetation Index

#Filtering the Modis Image Collection to the average of one month of data in 2019
NDVI = ee.ImageCollection('MODIS/006/MOD13A2').filter(ee.Filter.date('2019-04-01', '2019-04-30')).select("NDVI").mean()

# set some thresholds
vis_params = {
  'min': 0,
  'max': 9000,
  'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ],
}

colors = vis_params['palette']
vmin = vis_params['min']
vmax = vis_params['max']


# initialize our map
map1 = geemap.Map()
map1.centerObject(Brazil, 5)
map1.addLayer(NDVI.clip(Brazil), vis_params, 'NDVI Brazil') #clip it to Brazil, previously defined
map1.addLayer(NDVI.clip(Thailand), vis_params, 'NDVI Thailand') #clip it to Thailand, previously defined
map1.addLayer(NDVI.clip(schools), vis_params, 'NDVI Schools') #clip to schools, Brazil schools previously defined
map1.add_colorbar(vis_params, label="NDVI", layer_name="NDVI")
map1.addLayerControl()
map1

### Showing the change between 2014 and 2019 NDVI in country of choice

In [ ]:
#Choose if you want Brazil
country =  ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME', 'Brazil'))
#Choose if you want Thailand
# country = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME', 'Thailand'))

# set some thresholds
vis_params = {
  'min': 0,
  'max': 9000,
  'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ],
}

colors = vis_params['palette']
vmin = vis_params['min']
vmax = vis_params['max']

#Using MODIS NDVI, filtering to April-May of that year as that is time when vegetation often the highest
NDVI = ee.ImageCollection('MODIS/006/MOD13A2').filter(ee.Filter.date('2014-01-01', '2019-12-31')).select("NDVI").mean()
Modis_14 = ee.ImageCollection('MODIS/006/MOD13A2').filter(ee.Filter.date('2014-04-01', '2014-05-01')).select("NDVI").mean()
Modis_19 = ee.ImageCollection('MODIS/006/MOD13A2').filter(ee.Filter.date('2019-04-01', '2019-05-01')).select("NDVI").mean()
# create a split panel map
left_layer = geemap.ee_tile_layer(Modis_14.clip(country), vis_params,
                                  'VIIRS-DNB 2014', opacity=0.75)
right_layer = geemap.ee_tile_layer(Modis_19.clip(country), vis_params,
                                   'VIIRS-DNB 2019', opacity=0.75)

map3 = geemap.Map()
map3.centerObject(country, 5)
# map3.add_basemap('SATELLITE')
map3.split_map(left_layer=left_layer, right_layer=right_layer)
map3.add_colorbar(vis_params, label="NDVI", layer_name="NDVI")
map3.addLayerControl()
map3

### Showing the rate of change

In [ ]:
#Visualizing the rate of change from 2014 to 2020
NDVI = ee.ImageCollection('MODIS/006/MOD13A2').filter(ee.Filter.date('2014-01-01', '2019-12-31')).select('NDVI').mean()

# # sort by image "time_end"
# first_img = NDVI.sort('system:time_end').first()

# # reverse sort so that last=first
# last_img = NDVI.sort('system:time_end',False).first()

# # get rate of change (diff over # months: 73)
# NDVI_slope = (last_img.select('NDVI').subtract(first_img.select('NDVI'))).divide(73).clip(roi)

#Using MODIS NDVI, filtering to April-May of that year as that is time when vegetation often the highest
Modis_14 = ee.ImageCollection('MODIS/006/MOD13A2').filter(ee.Filter.date('2014-04-01', '2014-05-01')).mean()
Modis_19 = ee.ImageCollection('MODIS/006/MOD13A2').filter(ee.Filter.date('2019-04-01', '2019-05-01')).mean()

# get rate of change (diff over # years: 6)
NDVI_slope_year = (Modis_19.select('NDVI').subtract(Modis_14.select('NDVI'))).divide(6).clip(schools) #clipping to school buffer zones

In [ ]:
#change min, max to reflect the actual stats
vis_params = {'min':-400,
             'max':400,
             'palette':['1d4877','1b8a5a','f68838','ee3e32']}

colors = vis_params['palette']
vmin = vis_params['min']
vmax = vis_params['max']


# make it opaque so we can see underlying basemap
# initialize our map
NDVIMap = geemap.Map()
NDVIMap.centerObject(schools, 6)
NDVIMap.addLayer(NDVI_slope_year, vis_params, 'NDVI Slope Yearly',opacity=.75)
NDVIMap.add_colorbar(vis_params, label="Rate of Change", layer_name="Rate of Change")
NDVIMap.addLayerControl()
NDVIMap

## NDVI if you wanted to scale it

In [ ]:
# MODIS MOD13Q1
modis = ee.ImageCollection('MODIS/006/MOD13Q1')
modis = modis.filterDate(ee.DateRange('2015-01-01','2019-12-01'))

# select EVI and NDVI
evi = modis.select('EVI')
ndvi = modis.select('NDVI')

def scale_factor(image):
# scale factor for the MODIS MOD13Q1 product
    return image.multiply(0.0001).copyProperties(image, ['system:time_start'])
# mapping function to multiply by the scale factor
scaled_evi = evi.map(scale_factor)
scaled_ndvi = ndvi.map(scale_factor)

In [ ]:
# mean NDVI in the Brazil
vis_params={'min': 0,'max': 1, 'palette': ['red', 'yellow','green']}
            
colors = vis_params['palette']
vmin = vis_params['min']
vmax = vis_params['max']
            
Map5 = geemap.Map()
Map5.centerObject(Brazil, 6)
Map5.addLayer(scaled_ndvi.mean().clip(Brazil), vis_params)
Map5.add_colorbar(vis_params, label="NDVI", layer_name="NDVI")
Map5